In [15]:
from math import log
import numpy as np

In [17]:
rainfall = np.array( (0., 0.0057, 0., 0., 0., 0., 0., 0.0189, 0.0057 ,0.0214 ,0.1121,    \
                      0.2172, 0.0082, 0., 0.0007, 0., 0.0007, 0., 0., 0. ) )
fog               = rainfall * 0.0
E_div_P           = 0.5
canopy_fraction   = 0.58
canopy_storage    = 0.05
trunk_storage     = 0.01
stemflow_fraction = 0.04  # i.e. trunk fraction

In [19]:
fog

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

Implementation of Gash model from the original Hawaii Water Budget Model source code:
```
    dpnet=0.d0
    dcanint=0.d0
    if(fogeff(ilu(ip)).gt.0.5.and.drf.gt.0.0001)then
      if(icim.eq.2)then
            if(ifm.eq.1)then
                ceint2=ceint(1)
            elseif(ifm.eq.2)then
                if(drfy+dfogy.ge.ceint(2))then
                    ceint2=ceint(3)
                elseif(drfy+dfogy.ge.ceint(4))then
                    ceint2=ceint(5)
                else
                    ceint2=1.
                endif
            elseif(ifm.eq.3)then
                ceint2=exp(ceint(6)*(drfy+dfogy))
            else
                ceint2=1.
            endif
        Psat=-(cancap(ilu(ip))/(canfrac(ip)*cerf(ip)))*
1               log((1-cerf(ip))/(1-(1-ceint2)*cerf(ip)))
        if(drf+dfog.lt.Psat)then
           dcanint=canfrac(ip)*(drf+dfog)
        elseif(drf+dfog.gt.tcap(ilu(ip))/tfrac(ip))then
           dcanint=canfrac(ip)*Psat+canfrac(ip)*cerf(ip)*
1                     (drf+dfog-Psat)+tcap(ilu(ip))
        else
           dcanint=canfrac(ip)*Psat+canfrac(ip)*cerf(ip)*
1                     (drf+dfog-Psat)+tfrac(ip)*(drf+dfog)
        endif
        dcanint=gashcal*dcanint
        dpnet=drf+dfog-dcanint
      elseif(icim.eq.3)then
        dpnet=(drf+dfog)*(constAC*exp(constBD/(drf+dfog)))
        dcanint=drf+dfog+dpnet
      else
        dpnet=frg(m,irfzone(ip),jfr(m,irfzone(ip),i),k)*pnet
      endif
    else
      dpnet=drf+dfog
    endif
```

The Oahu input files containing interception coefficients (`gash.prn`) looks like this:
```
       1       1.0       0       0       0       0       0      1.0
```
The HWB Gash method code in the Oahu case is '1' (`ifm` in the code snippet above), and the constant 'a' has a value of 1.0 (`ceint(1)` in code snippet above).

In [10]:
def precip_at_saturation(canopy_capacity, canopy_fraction, canopy_E_to_P):
    Psat = -(canopy_capacity / (canopy_fraction * canopy_E_to_P)) * log(1.0 - canopy_E_to_P)
    return Psat

In [28]:
def gash_interception(canopy_fraction, Psat, canopy_E_to_P, rainfall, fog, trunk_fraction, trunk_capacity):
    
    rainfall_plus_fog = rainfall + fog
    
    if rainfall_plus_fog < Psat:
        can_int = canopy_fraction * rainfall_plus_fog 
    elif rainfall_plus_fog > (trunk_capacity / trunk_fraction):
        can_int = canopy_fraction * Psat + canopy_fraction * canopy_E_to_P * (rainfall_plus_fog - Psat)   \
                  + trunk_capacity
    else:
        can_int = canopy_fraction * Psat + canopy_fraction * canopy_E_to_P * (rainfall_plus_fog - Psat)   \
                  + trunk_fraction * rainfall_plus_fog
    
    return can_int

In [29]:
Psat = precip_at_saturation(canopy_storage,canopy_fraction,E_div_P)

In [35]:
interception =  [gash_interception( canopy_fraction, Psat, E_div_P, P, F, stemflow_fraction, trunk_storage ) for P,F in zip(rainfall,fog)]

In [36]:
interception

[0.0,
 0.003306,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.010962,
 0.003306,
 0.012411999999999998,
 0.06501799999999999,
 0.10633335902799726,
 0.004756,
 0.0,
 0.00040599999999999995,
 0.0,
 0.00040599999999999995,
 0.0,
 0.0,
 0.0]